# Import Library

In [1]:
import os
import re
import sys
import timeit
import json
import pymupdf
import psycopg2
import pandas as pd
from pandas import DataFrame
from pathlib import Path
import requests
import logging
import os
import datetime
import time
import psutil
from io import StringIO
from setuptools.namespaces import flatten
from IPython.display import clear_output

In [2]:
print(sys.path)

['/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python313.zip', '/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13', '/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload', '', '/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF/.venv/lib/python3.13/site-packages', '/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF/.venv/lib/python3.13/site-packages/setuptools/_vendor']


In [3]:
!ls
os.getcwd()

core                       requirements.txt
libraries.txt              shipping_label.csv
pdf-parsing-notebook.ipynb task-script


'/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF'

In [4]:
file_path = '/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF/shipping_label.csv'

In [5]:
df = pd.read_csv(file_path)
df_0 = df.drop(['inserted_at'], axis = 1)
df_0

,platform_name,delivery_platform,delivery_tracking_number,order_id,order_no,external_order_no,shipping_label
0,Shopee,Giao Hàng Nhanh,G8PG68DW,16033053,SPEJNJ-240423GDTCSH56,SPEJNJ-240423GDTCSH56,https://files.onpoint.vn/order/2024/04/23/8667...
1,Shopee,SPX Express,SPXVN047044006034,16032662,SPEJNJ-240423GBDEC0XM,SPEJNJ-240423GBDEC0XM,https://files.onpoint.vn/order/2024/04/23/1869...
2,Shopee,Giao Hàng Nhanh,G8PG9QNK,16032417,SPECSC-240423GA23PYAJ,SPECSC-240423GA23PYAJ,https://files.onpoint.vn/order/2024/04/23/5646...
3,Shopee,Giao Hàng Nhanh,G8PG9PX4,16032195,SPECSC-240423G92X6K4G,SPECSC-240423G92X6K4G,https://files.onpoint.vn/order/2024/04/23/8805...
4,Shopee,Giao Hàng Nhanh,G8PG7HWG,16031437,SPEUIM-240423G697MDT4,SPEUIM-240423G697MDT4,https://files.onpoint.vn/order/2024/04/23/1101...
...,...,...,...,...,...,...,...
995,Shopee,Giao Hàng Nhanh,G8DTBU3T,15695256,SPESDP-240411FD6M6KUT,SPESDP-240411FD6M6KUT,https://files.onpoint.vn/order/2024/04/11/4516...
996,Shopee,SPX Express,SPXVN041469733864,15695249,SPEBRT-240411FD598RCG,SPEBRT-240411FD598RCG,https://files.onpoint.vn/order/2024/04/11/2137...
997,Shopee,SPX Express,SPXVN043318599284,15695234,SPECBB-240411FD1VHHDU,SPECBB-240411FD1VHHDU,https://files.onpoint.vn/order/2024/04/11/4993...
998,Shopee,Giao Hàng Nhanh,G8DTDPWU,15695233,SPEJNJ-240411FD1E7V87,SPEJNJ-240411FD1E7V87,https://files.onpoint.vn/order/2024/04/11/3749...


In [6]:
order_id_items = []
items = list(df_0['order_no'])
for item in items:
    order_item = item.split('-')[-1]
    order_id_items.append(order_item)

In [7]:
df_0.count()

platform_name               1000
delivery_platform           1000
delivery_tracking_number    1000
order_id                    1000
order_no                    1000
external_order_no           1000
shipping_label              1000
dtype: int64

In [8]:
file_list = list(df['shipping_label'])

In [9]:
df_1 = pd.DataFrame(list(zip(order_id_items, file_list)), columns=['order_id', 'file_url'])
dict_1 = df_1.to_dict(orient='records')
dict_2 = {x['file_url'].split('/')[-1]:x['order_id'] for x in dict_1}
dict_3 = {x['file_url'].split('/')[-1]:x['file_url'] for x in dict_1}


# Download PDFs from list of URLs

In [10]:
start = timeit.default_timer()
count = 0
for file in file_list:
    file_name = file.split('/')[-1]
    day = file.split('/')[6]
    month = file.split('/')[5]
    year = file.split('/')[4]
    response = requests.get(file)
    path = Path(f'/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF/Shopee/{year}/{month}/{day}')
    path.mkdir(parents=True, exist_ok=True)
    with open(f'/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF/Shopee/{year}/{month}/{day}/' + file_name, 'wb') as f:
      clear_output()
      count = count + 1
      f.write(response.content)
      print(f'Downloading progess: {count}/{len(file_list)} - {count*100.0/len(file_list)}%')    
stop = timeit.default_timer()
runtime = str(datetime.timedelta(seconds=(stop - start)))
print('Download time:', runtime)

#Resource Monitor
pid = os.getpid()
python_process = psutil.Process(pid)
memory_use = python_process.memory_info()[0]/2.**30
cpu_use = python_process.cpu_percent
print('Memory Use:', memory_use, 'Gb')
print('CPU Use:', cpu_use)

Download time: 0:05:23.157668
Memory Use: 0.1083831787109375 Gb
CPU Use: <bound method Process.cpu_percent of psutil.Process(pid=39243, name='Python', status='running', started='10:56:33')>


# Extract Data from PDFs

In [11]:
start = timeit.default_timer()
result = []
count = 0
for year in os.listdir(Path(f'/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF/Shopee/')):
  for month in os.listdir(Path(f'/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF/Shopee/{year}')):
    for day in os.listdir(Path(f'/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF/Shopee/{year}/{month}')):
      for file_name in os.listdir(Path(f'/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF/Shopee/{year}/{month}/{day}')):
        clear_output()
        count = count + 1
        print(f'Processing progess: {count}/{len(file_list)} - {count*100.0/len(file_list)}%')

        file_result = {'Billing_date': f'{day}/{month}/{year}', 'order_id':dict_2[file_name]}
        with pymupdf.open(f'/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF/Shopee/{year}/{month}/{day}/{file_name}') as doc:
          cells = []
          for page in doc:
            for table in page.find_tables().tables:
              for tb in table.extract(strategy="lines_strict"):
                for cell in list(filter(lambda x: x and x != "", tb)):
                  cells.append(cell)

        for cell in cells:
          file_result['file_url'] = dict_3[file_name]
          if 'Mã đơn hàng:' in cell:
            file_result['OrderID'] = cell.split(': ')[-1]
          if "Đến:" in cell:
            receiver = cell.split('\n')
            match_phone_no = re.search(r"[^a-z A-Z,]+", receiver[2]).group()
            if receiver[2] == match_phone_no:
              file_result['phone_number'] = match_phone_no
              receiver_address = " ".join(receiver[3:]).split(", ")
            else:
              file_result['phone_number'] = '-'
              receiver_address = " ".join(receiver[2:]).split(", ")
            file_result['receiver_name'] = receiver[1]
            file_result['receiver_address'] = {
                'street': ', '.join(receiver_address[0:-3]),
                'ward': receiver_address[-3],
                'district': receiver_address[-2],
                'city': receiver_address[-1]
            }
        result.append(file_result)

stop = timeit.default_timer()
extract_runtime = str(datetime.timedelta(seconds=(stop - start)))
print('Extract time:', extract_runtime)

#Resource Monitor
pid = os.getpid()
python_process = psutil.Process(pid)
memory_use = python_process.memory_info()[0]/2.**30
cpu_use = python_process.cpu_percent
print('Memory Use:', memory_use, 'Gb')
print('CPU Use:', cpu_use)

Processing progess: 1000/1000 - 100.0%
Extract time: 0:00:49.875614
Memory Use: 0.1511993408203125 Gb
CPU Use: <bound method Process.cpu_percent of psutil.Process(pid=39243, name='Python', status='running', started='10:56:33')>


# Export raw data to JSON

In [ ]:
raw_data = json.dumps(result, indent = 2)
with open('raw_data.json', 'w', encoding ='utf-8') as f:
    f.write(raw_data)

# Transform Data

In [ ]:
try:
    monitor_time = str(datetime.datetime.now())
    created_at = str(datetime.datetime.now())
    updated_at = str(datetime.datetime.now())
    platform = 'SHOPEE'
    final = []
    f = open('/Users/mac/Documents/Documents - Minh Nguyen/GitHub/PDF-Label-Parsing-with-PyMuPDF/raw_data.json')
    records = json.load(f)
    for record in records:
        record = {
            'order_id': record.get('order_id'),
            'label_order_id': record.get('OrderID'),
            'platform': platform,
            'file_url': record.get('file_url'),
            'billing_date': record.get('Billing_date'),
            'receiver_name' : record.get('receiver_name'),
            'phone_number': record.get('phone_number'),
            'street_name': record.get('receiver_address').get('street'),
            'ward': record.get('receiver_address').get('ward'),
            'district': record.get('receiver_address').get('district'),
            'city': record.get('receiver_address').get('city'),
            'monitor_time' : monitor_time,
            'created_at': created_at,
            'updated_at': updated_at
        }
        final.append(record)
except Exception as e:
    logging.exception(e)

# Export transformed data to JSON

In [ ]:
raw_data = json.dumps(final, indent = 2)
with open('final.json', 'w', encoding ='utf-8') as f:
    f.write(raw_data)

In [ ]:
df_final = pd.DataFrame(final)
df_final

# Load into DB

In [ ]:
conn = psycopg2.connect(database=DB_NAME,
                        user=DB_USER,
                        password=DB_PASS,
                        host=DB_HOST,
                        port=DB_PORT)

print('Successfully connect to database!')

start_time = time.time()
sio = StringIO()
df_final.to_csv(sio, index=None, header=None)
sio.seek(0)
with conn.cursor() as c:
    c.copy_expert(
        sql="""
        DELETE FROM athena_warehouse.spe_shipping_label_address_parse;

        COPY athena_warehouse.spe_shipping_label_address_parse (
            order_id,
            label_order_id,
            platform,
            file_url,
	        billing_date,
	        receiver_name,
            phone_number,
	        street_name,
	        ward,
	        district,
	        city,
	        monitor_time,
	        created_at,
	        updated_at
            ) FROM STDIN WITH CSV""",
        file=sio
    )
    conn.commit()

end_time = time.time()
total_time = end_time - start_time
print(f"Insert time: {total_time} seconds")